In [1]:
import cv2 
from scipy.misc import imresize 
import moviepy.editor as moviepy
from tqdm import tnrange, tqdm_notebook

import AsciiMosaic #cvts all chars for pixel values 
import Text2Img #cvts specifics chars from text file to values 

# import importlib
# importlib.reload()

In [2]:
def translate(value, leftMin, leftMax, rightMin, rightMax):
    '''a function to map values from one domain to another
    for converting the iteration index to a parameter for the crop selection'''
    # Figure out how 'wide' each range is
    leftSpan = leftMax - leftMin
    rightSpan = rightMax - rightMin

    # Convert the left range into a 0-1 range (float)
    valueScaled = float(value - leftMin) / float(leftSpan)

    # Convert the 0-1 range into a value in the right range.
    return rightMin + (valueScaled * rightSpan)

In [ ]:
size = [1080, 1920]

video_in = 'obama.mp4'
video_out = 'tester.avi'
orig_movie = moviepy.VideoFileClip(video_in) 

fourcc = cv2.VideoWriter_fourcc(*'XVID') #codec 
writer = cv2.VideoWriter(video_out, fourcc, orig_movie.fps, tuple(size[::-1]), False)

mosiac = Text2Img.Processor(10) #choose your processor 

In [ ]:
scale = 100
min_scale = 20
min_dims = [int(x/2*min_scale/100) for x in size]
shift = [0, 0]

'''grab frame from video, process, resize, write to output'''
for i in tnrange(int(orig_movie.fps * orig_movie.duration), desc='progress'): #
    
    #grab the next frame and process it 
    frame = orig_movie.get_frame(i/orig_movie.duration)
    ascii_img = mosiac.process_image(imresize(frame, 200))
    center = [int(x/2) for x in ascii_img.shape]
    
    #adjust frame parameters 
    if i <= 200: 
        '''scale down and push to left corner'''
        scale = int(translate(i, 0, 200, 100, 20))
        shift[0] = int(translate(i, 0, 200, 0, -(size[0]/2 - min_dims[0])))
        shift[1] = int(translate(i, 0, 200, 0, -(size[1]/2 - min_dims[1])))
        
    elif i > 600 and i < 700: 
        '''keep scale and move to right corner'''
        shift[1] = int(translate(i, 600, 700, -(size[1]/2 - min_dims[1]), size[1]/2 - min_dims[1]))
        
    if i > 700 and i < 900: 
        '''scale up and move to origin'''
        scale = int(translate(i, 700, 900, 20, 100))
        shift[0] = int(translate(i, 700, 900, -(size[0]/2 - min_dims[0]), 0))
        shift[1] = int(translate(i, 700, 900, (size[1]/2 - min_dims[1]), 0))

    #crop the selection determine by the above parameters 
    new_loc = [sum(x) for x in zip(center, shift)]
    rect_size_x, rect_size_y = [int(x/2*scale/100) for x in size]

    cropped = ascii_img[(new_loc[0] - rect_size_x):(new_loc[0] + rect_size_x),
                   (new_loc[1] - rect_size_y):(new_loc[1] + rect_size_y)]
              
    writer.write(imresize(cropped, size).astype('u1'))
    
writer.release()

A Jupyter Widget

In [ ]:
#for stopping early 
writer.release()
cv2.imwrite('test.png', ascii_img)

In [ ]:
# '''grab frame from video, process, resize, write to output'''
# for i in tnrange(int(orig_movie.fps * orig_movie.duration), desc='progress'): #
#     frame = orig_movie.get_frame(i/orig_movie.duration)
#     ascii_img = mosiac.process_image(imresize(frame, 1))
#     writer.write(imresize(ascii_img, SIZE).astype('u1'))
    
# writer.release()

In [ ]:
# writer.release()
# cv2.imwrite('test.png', ascii_img)

## Pixels to Ascii Characters

This code is inspired from the [BitsOfCode](https://bitesofcode.wordpress.com/2017/01/19/converting-images-to-ascii-art-part-1/) blog post on converting pixel values of an image to ascii characters. I created an image processor in [AsciiMosaic.py](AsciiMosaic.py) to do this efficiently, making use of numpy's `np.vectorize` function to vectorize iterating through the image and mapping the pixels to ascii characters. A rough workflow is explained in [test_ascii_mosiac_gif.ipynb](test_ascii_mosiac_gif.ipynb), where I convert an image to the ascii characters, and create a gif to zoom in on the detail. 

<img src="obama/obama_ascii.gif" width="600" alt="raw" />

I decided to go one step further and select specific text for the ascii character mapping. This was accomplished by getting the transcript of a speech and selecting the intensity of the background to match that of the pixel which is behind it. The process was actually simpler than the random character mapping, but maybe that is because I had solved random mapping already. The processor code is under [Text2Img.py](Text2Img.py). I also wrote a few scripts to zoom in on the detail for the output video. 

## VIDEO HERE